In [ ]:
from ipyparallel import Cluster
c = await Cluster(engines="mpi").start_and_connect(n=4, activate=True)

In [ ]:
%%px
from mpi4py import MPI
comm=MPI.COMM_WORLD

from ngsolve import *
from ngsolve.webgui import Draw
from netgen.occ import *

from NavierStokesSIMPLE import NavierStokes
ngsglobals.msg_level = 0

In [ ]:
%%px

shape = Rectangle(2,0.41).Circle(0.2,0.2,0.05).Reverse().Face()
shape.edges.name="wall"
shape.edges.Min(X).name="inlet"
shape.edges.Max(X).name="outlet"
# Draw (shape);

mesh = OCCGeometry(shape, dim=2).GenerateMesh(maxh=0.07, comm=comm)
for l in range(2):
    mesh.Refine()
mesh = Mesh(mesh)
mesh.Curve(3)
if comm.rank==1:
    Draw (mesh);

In [ ]:
%%px

order=2
nu=0.001
inflow="inlet"
outflow="outlet"
wall="wall"


V = HDiv(mesh, order=order, dirichlet=inflow+"|"+wall, RT=False)
Vhat = TangentialFacetFESpace(mesh, order=order-1, dirichlet=inflow+"|"+wall+"|"+outflow)
Sigma = HCurlDiv(mesh, order = order-1, orderinner=order, discontinuous=True)
if mesh.dim == 2:
    S = L2(mesh, order=order-1)            
else:
    S = VectorL2(mesh, order=order-1)

Sigma.SetCouplingType(IntRange(0,Sigma.ndof), COUPLING_TYPE.HIDDEN_DOF)
Sigma = Compress(Sigma)
S.SetCouplingType(IntRange(0,S.ndof), COUPLING_TYPE.HIDDEN_DOF)
S = Compress(S)
        
X = V*Vhat*Sigma*S
for i in range(X.ndof):
    if X.CouplingType(i) == COUPLING_TYPE.WIREBASKET_DOF:
        X.SetCouplingType(i, COUPLING_TYPE.INTERFACE_DOF)
        
u, uhat, sigma, W  = X.TrialFunction()
v, vhat, tau, R  = X.TestFunction()

if mesh.dim == 2:
    def Skew2Vec(m):
        return m[1,0]-m[0,1]
else:
    def Skew2Vec(m):   
        return CF( (m[0,1]-m[1,0], m[2,0]-m[0,2], m[1,2]-m[2,1]) )

dS = dx(element_boundary=True)
n = specialcf.normal(mesh.dim)
def tang(u): return u-(u*n)*n
        
stokesA = -0.5/nu * InnerProduct(sigma,tau) * dx + \
          (div(sigma)*v+div(tau)*u) * dx + \
          (InnerProduct(W,Skew2Vec(tau)) + InnerProduct(R,Skew2Vec(sigma))) * dx + \
          -(((sigma*n)*n) * (v*n) + ((tau*n)*n )* (u*n)) * dS + \
          (-(sigma*n)*tang(vhat) - (tau*n)*tang(uhat)) * dS


a = BilinearForm (X, eliminate_hidden = True)
a += stokesA
a += 10*nu*div(u)*div(v) * dx
a.Assemble()

In [ ]:
%%px
uin=CF( (1.5*4*y*(0.41-y)/(0.41*0.41), 0) )
gf = GridFunction(X)
gfu,gfuhat,gfsigma,gfS = gf.components                   
gfu.Set(uin, definedon=mesh.Boundaries(inflow))

In [ ]:
# inva = a.mat.Inverse(X.FreeDofs(), inverse="masterinverse")
# gfu.vec.data -= inva@a.mat * gfu.vec

In [ ]:
%%px
from ngsPETSc import pc, KrylovSolver

solver = KrylovSolver(a,X, solverParameters={'ksp_type': 'cg', 'pc_type': 'gamg', \
                                             'ksp_monitor': '', 'ksp_rtol':1e-8})

f = LinearForm(X).Assemble()
f.vec.data = -a.mat*gfu.vec

# gfucorr = solver.solve(f)
# gfu.vec.data += gfucorr.vec

In [ ]:
%%px
from ngsPETSc import pc
from ngsolve.krylovspace import CGSolver

pre = Preconditioner(a, "PETScPC", pc_type="gamg")
printing = '\r' if comm.rank==0 else None
invA = CGSolver(a.mat, pre, printrates=printing, maxiter=1000)
gf.vec.data -= invA@a.mat * gf.vec

In [ ]:
%%px
if comm.rank > 0:
    Draw (gfu);